# Switched Networks

To start our discussion of routing between hosts, we'll examine switched networks.

Remember an internet in an interconnected network of switched networks.

## Bridges

A bridge is device that connects to groups of hosts:

<br>
<img src="images/18-simple-bridge.png" width="400">
<br>

The links on either side of the bridge are shared.

If A sends a message destined to B, B will hear it because it is connected to the shared link. The job of the bridge is to decide whether to broadcast that message to the other link or not.

A simple bridge keeps a table of the hosts connected to either port:

```
Host | Port
-----------
 A   |  1
 B   |  1
 C   |  1
 X   |  2
 Y   |  2
 Z   |  2
```

With this table, the bridge can always decide whether it needs to forward a packet to the other port.

## Building the Table

For very small static networks the table could be hard coded.

For dynamic networks, the table is learned as packets are received.

It starts out empty. When the bridge receives a packet on one of its ports, it added that host and port to its table.

Every entry has a Time To Live (TTL). If TTL expires and no new messages have been received from that host, its entry is removed.

## A More Complicated Switched Network

Many switched netwoks consist of a switches and hosts connected to them.



<br>
<img src="images/19-learning-bridges.png" width="400">
<br>

Loops in networks like this can cause packet to get indefinitely stuck within them.

We want to configure the network so that it behaves as if there were no cycles. Every switch will have preferred links that it will use so that resulting network graph is acyclic.

<br>
<img src="images/20-learning-bridges-spanning.png" width="400">
<br>

The enabled edges for a spanning tree of this graph.

At network boot every switch needs to decide what ports to use so that the overall network forms a spanning tree.

No switch has a global view of the network, but they all must agree together on the links to use.

A distributed algorithm is used to do this.

What needs to be decided?

- who the root node is
- which preferred ports to use to send packets towards the root

## A Distributed Algorithm for Navigating a Switched Network

To agree on the root and preferred ports, all switches exchange configuration messages.

Configuration Messages contain:
- The ID of the switch sending the message
- The ID of the switch the sender thinks is the root
- The distance in hops from the sender to the root

At initialization, every switch sends a message to its neighbors claiming that it is the root and that it is 0 hops away from the root.

At the end, the switch with the smallest ID will be root of the network.

Every switch keeps track of three pieces of information:
- Who the root is
- How far away the root is
- Which port to use to send messages to the root

When a switch received a new configuration message, it will update its configuration if:

- The messages identifies a root with a smaller ID
    - Update who the root is and how far away the root it
- The message identifies the same root but the sender is closer to the root
    - Update the port to use to get to the root and the distance to the root
- The message that identifies the same root and distance, but the sender has a smaller ID that the current preferred port
    - Update the preferred port to go through the switch with the smaller ID

After a switch is no longer the root, it does not originate messages, it only forwards along configuration messages received. When the network has converged to a spanning tree, only the root will be sending out messages.

If a switch goes down, one of its neighbors will no longer be receiving messages from the root.

When this happens, that switch will assume that the root has gone down and will start broadcasting that it is the root again, starting the process over again and forcing the network to converge to a new spanning tree.

<br>
<img src="images/20-learning-bridges-spanning.png" width="400">
<br>

The algorithm from S3's perspective:

Message Format: (Sender ID, Root ID, distance from sender to root)

1) S3 receives (S5, S5, 0). S3 will not update its configuration since it has the smaller ID 
2) S3 received (S2, S2, 0). S3 accepts S2 as the root, update its distance to be 1, and updates the preferred port to be S2
3) Meanwhile, S2 accepts S1 as the root and sends (S2, S1, 1). S3 will accept this, update its distance to be 2, and record S1 as the root
4) S5 also accepts S1 and forwards along (S5, S1, 1). S3 doesn't accept this since its preferred port to S2 is a smaller ID than S5.